In [1]:
import model as m
from sklearn.linear_model import LogisticRegression
clf = m.Classifier(1,LogisticRegression())

In [20]:
#stop word sets specified for this data set
path = "../data/stop_words_news_groups/"
with open(path+"stop_words_no_numbers.txt", 'r') as content_file:
        content = content_file.read().replace(" ","").replace("\n","").split(",")
        stop_words_no_nums = frozenset(content)
with open(path+"stop_words_with_nums.txt", 'r') as content_file:
        content = content_file.read().replace(" ","").replace("\n","").split(",")
        stop_words_with_nums = frozenset(content)


frozenset({'23',
           '1952',
           '283',
           'yet',
           '985',
           '506711',
           '140195',
           '39753',
           '3750',
           '03000',
           '0065',
           '4014',
           '206264',
           '904',
           '4019',
           '1360',
           '7224',
           'keep',
           '11544',
           '8721',
           '1063',
           '77030',
           '01890',
           '33444',
           'further',
           '314',
           '303',
           '336',
           '770',
           '08544',
           'although',
           '680020',
           '92057',
           '593',
           '4892',
           '855',
           '12500',
           '10000001',
           '1506',
           '2425',
           '1459',
           '333',
           'somewhere',
           '460',
           '40950',
           '552',
           '00011110',
           '0542',
           '95014',
           '044',
           '4563',
        

In [2]:
params_trial_1 = { 
            #penalty is l2 by default
            'vect__min_df': ([0.0,0.1,0.2,0.25,0.30]),
            'vect__max_features': (1000, 5000,10000, 50000),
            'vect__ngram_range': ((1, 1), (1, 2)),
            'tfidf__use_idf': (False,True),
            'tfidf__norm': ('l1','l2',None),
            'clf__max_iter': (50, 100, 200,500),
            'clf__solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
            'clf__alpha': (0.1,0.01,0.0001,0.000001,0.00000001),
            'clf__C':(0.1,0.01,0.0001,0.000001,0.00000001),
            }

In [ ]:
#### TEST 1 ####
clf.fit(params_trial_1,5)

In [ ]:
clf.eval_on_test([],False)
clf.learning_curve([0.2,0.4,0.6,0.8,1.0])